<h2> Loading MNIST dataset <h2>

In [19]:
from sklearn.datasets import fetch_openml
# Preferred: fetch by name
X, y = fetch_openml("mnist_784", version=1, return_X_y=True)
print(X.shape)
print(y.shape)

(70000, 784)
(70000,)


<h2> Train test and validation split <h2>

In [2]:
from sklearn.model_selection import train_test_split


# First split: Train + Test
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=10000, random_state=42
)

# Second split: Train + Validation
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=10000, random_state=42
)

print("Train:", X_train.shape)
print("Validation:", X_valid.shape)
print("Test:", X_test.shape)



Train: (50000, 784)
Validation: (10000, 784)
Test: (10000, 784)


<h2> Importing libraries that will be used across models <h2>

In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

(50000, 784)


<h2> Defining the true labels <h2>

In [13]:
true_labels = y_valid

<h2> Training a ridge classifier <h2>

In [ ]:
from sklearn.linear_model import RidgeClassifier

steps_ridge = Pipeline([
   ( 'scaler',StandardScaler()),
   ('classifier',RidgeClassifier(alpha=0.5))
])

steps_ridge.fit(X_train,y_train)

predictions = steps_ridge.predict(X_valid)


print("Accuracy score of Ridge regression is ",accuracy_score(predictions,true_labels))


Accuracy score of Ridge regression is  0.8455


<h2> Training SVM classifier <h2>

In [ ]:
from sklearn.svm import SVC


steps_SVM = Pipeline([
    ('scaler',StandardScaler()),
    ('classifier',SVC(kernel='linear'))
])


steps_SVM.fit(X_train,y_train)

predictions = steps_SVM.predict(X_valid)

print("Accuracy score of SVM r is ",accuracy_score(predictions,true_labels))



Accuracy score of SVM r is  0.921


<h2> Training a random forest classifier <h2>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

forest_classifier = RandomForestClassifier()

forest_classifier.fit(X_train,y_train)
predictions = forest_classifier.predict(X_valid)
true_labels = y_valid
print("Accuracy score for Tuned Random Forest is " ,accuracy_score(predictions,true_labels))




Accuracy score for Tuned Random Forest is  0.9696


<h2> Training hard Voting ensemble model consisting  the three models <h2>

In [ ]:
from sklearn.ensemble import VotingClassifier

models =[
    ('ridge',steps_ridge),
    ('svm',steps_SVM),
    ('forest',forest_classifier)

]

ensemble_hard = VotingClassifier(
    estimators = models,
    voting='hard'
)

ensemble_hard.fit(X_train,y_train)

predictions = ensemble_hard.predict(X_valid)

print("Accuracy score for the hard voting classifier is")
print(accuracy_score(predictions,true_labels))




Accuracy score for the hard voting classifier is
0.9412


<h2> Stacked ensemble <h2>

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression


final_estimator = LogisticRegression()

stacked_model = StackingClassifier(
    estimators=models,
    final_estimator=final_estimator
)

stacked_model.fit(X_train,y_train)

predictions = stacked_model.predict(X_test)

print("The Accuracy score for Stacked ensemble is" , accuracy_score(predictions,true_labels))

/Users/parvizpirizade/Documents/DecisionTrees/venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


The Accuracy score for Stacked ensemble is 0.0984
